In [7]:
import time
import random
import datetime
# dc_crawling.py 같은 폴더에 있다고 가정
#from dc_crawling import *
#from stock import *
from kafka import KafkaProducer

In [9]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

### 안쓰는 부분 주석처리

In [10]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

# def crawl(code, time):
#     headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

#     date = time[:8]
    
#     result = list()
#     pages = range(1, 41)
#     for page in pages:
#         url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, time, page)
#         resp = download(url, headers = headers)
#         dom = BeautifulSoup(resp.content, 'lxml')
        
#         result.extend([(date, t, p) for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])])
#         print(result)
#     return result

In [ ]:
# codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', '041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', '008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', '089890', '030350', '230980', '069410'}


# data = {i:[] for i in codes}

# now = datetime.datetime.now()
# year = now.year
# month = now.month
# day = now.day

# now = datetime.datetime.now()

# for i in tqdm(range(1)):
    
    
#     time = datetime.datetime(now.year, now.month, now.day, 16, 11, 22).strftime("%Y%m%d%H%M%S")
    
#     if now.weekday() == 5 or now.weekday() == 6:
#         now = now - datetime.timedelta(days = 1)
#         continue
    
   
#     for code in codes:
#         data[code] = crawl(code, time)
#     now = now - datetime.timedelta(days = 1)

In [13]:
# bootstrap_servers = ['localhost:9092','localhost:9093']  # kafka broker ip
# topicName = 'realtime_stock'
# producer = KafkaProducer(bootstrap_servers = bootstrap_servers)

codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', 
'041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', 
'008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', 
'089890', '030350', '230980', '069410'}

now = datetime.datetime.now()
    
############ 
# {'code': '336260', 'time':'2022.01.07 14:43', 'price':'127,500'}
#############

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# ticker 처음부터 다 돌기
for code in codes:
    time = datetime.datetime(now.year, now.month, now.day)

    if now.weekday() == 5 or now.weekday() == 6:
        now -= datetime.timedelta(days = 1)
        continue
    
    result = list()
    pages = range(1, 13)
    for page in pages:
        url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, time.strftime("%Y%m%d")+"161122", page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        
        # result = for t, p in zip([i.text for i in dom.select('tr > td:nth-of-type(1) > span')], [i.text for i in dom.select('tr > td:nth-of-type(2) > span')])}
        for i in range(len(dom.select('tr > td:nth-of-type(2) > span'))):
            # result list에 dictionary 형태로 추가
            result.append({
                'code':code, 
                'date': time.strftime("%Y.%m.%d") + " " + dom.select('tr > td:nth-of-type(1) > span')[i].text,
                'price': dom.select('tr > td:nth-of-type(2) > span')[i].text
                }) 

# 날짜 왜 깎는지 잘 모르게씀..!           
now -= datetime.timedelta(days = 1)

{'code': '008770', 'time': '2022.01.07 15:55', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:52', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:51', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:50', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:46', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:45', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:44', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:41', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:40', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:30', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:19', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:18', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:17', 'price': '77,400'}
{'code': '008770', 'time': '2022.01.07 15:16', 'price': '77,300'}
{'code': '008770', 'time': '2022.01.07 15:15', 'price': '77,400'}
{'code': '

KeyboardInterrupt: 

In [6]:
dict('123', '14:05', '125,00')

TypeError: dict expected at most 1 argument, got 3